In [15]:
!pip install transformers==4.25.1
!pip install datasets==2.8.0
!pip install rouge
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re
import torch
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm
import glob
import json
import os

In [17]:
def load_json_data(path):

    with open(path) as f:
        data = json.load(f)

    ids = []
    dialogues = []
    summaries = []
    topic = []
    for datum in data["data"]:
        ids.append(datum["header"]["dialogueInfo"]["dialogueID"])

        prev_speaker_id = None
        prev_line = ""
        utts = []
        for dialogue in datum["body"]["dialogue"]:
            utterance = dialogue["utterance"].strip()

            if dialogue["participantID"] == prev_speaker_id:
                prev_line += " " + utterance
            else:
                if prev_line:
                    utts.append(prev_line)
                prev_line = utterance
                prev_speaker_id = dialogue["participantID"]
        if prev_line:
            utts.append(prev_line)

        dialogues.append(utts)
        summaries.append(datum["body"].get("summary"))

    for i in range(len(data['data'])):
      topic.append(data['data'][i]['header']['dialogueInfo']['topic'])
    return ids[:1555], dialogues[:1555], summaries[:1555], topic[:1555]

def data_load(filename, is_meta=False):
    ids_list, dialogues_list, summaries_list, topic_list = [], [], [], []
    dialogues_sep = []

    for file in tqdm(filename):
      ids, dialogues, summaries, topic = load_json_data(file)
      for id, text, summ, top in zip(ids, dialogues, summaries, topic):
        ids_list.append(id)
        if is_meta:
          text.insert(0,"#"+top+"#")
        dialogues_list.append(text)
        summaries_list.append(summ)
        topic_list.append(top)
    
    for text in tqdm(dialogues_list):
      dialogues_sep.append("[sep]".join(text))

    return ids_list, dialogues_sep, summaries_list


In [18]:
def load_json_data2(path):

    with open(path) as f:
        data = json.load(f)

    ids = []
    dialogues = []
    summaries = []
    topic = []
    for datum in data["data"]:
        ids.append(datum["header"]["dialogueInfo"]["dialogueID"])

        prev_speaker_id = None
        prev_line = ""
        utts = []
        for dialogue in datum["body"]["dialogue"]:
            utterance = dialogue["utterance"].strip()

            if dialogue["participantID"] == prev_speaker_id:
                prev_line += " " + utterance
            else:
                if prev_line:
                    utts.append(prev_line)
                prev_line = utterance
                prev_speaker_id = dialogue["participantID"]
        if prev_line:
            utts.append(prev_line)

        dialogues.append(utts)
        summaries.append(datum["body"].get("summary"))

    for i in range(len(data['data'])):
      topic.append(data['data'][i]['header']['dialogueInfo']['topic'])
    return ids[:194], dialogues[:194], summaries[:194], topic[:194]

def data_load2(filename, is_meta=False):
    ids_list, dialogues_list, summaries_list, topic_list = [], [], [], []
    dialogues_sep = []

    for file in tqdm(filename):
      ids, dialogues, summaries, topic = load_json_data2(file)
      for id, text, summ, top in zip(ids, dialogues, summaries, topic):
        ids_list.append(id)
        if is_meta:
          text.insert(0,"#"+top+"#")
        dialogues_list.append(text)
        summaries_list.append(summ)
        topic_list.append(top)
    
    for text in tqdm(dialogues_list):
      dialogues_sep.append("[sep]".join(text))

    return ids_list, dialogues_sep, summaries_list


In [19]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@,-\[\]\(\)]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

In [20]:
dirname = "/content/drive/MyDrive/인공지능/아이펠톤/PoC/kt_data/Training"
filenames = os.listdir(dirname) 
train_full_filename = []

for filename in filenames:
    fn = os.path.join(dirname, filename)
    if dirname + '/.ipynb_checkpoints' != fn:
        train_full_filename.append(fn)

dirname2 = "/content/drive/MyDrive/인공지능/아이펠톤/PoC/kt_data/Validation"
filenames2 = os.listdir(dirname2) 
val_full_filename = []

for filename in filenames2:
    fn2 = os.path.join(dirname2, filename)
    if dirname + '/.ipynb_checkpoints' != fn2:
        val_full_filename.append(fn2)

In [21]:
train_ids, train_dialogues, train_summaries = data_load(train_full_filename, is_meta=True)
val_ids, val_dialogues, val_summaries = data_load2(val_full_filename)

train_texts = data_process(train_dialogues)
val_texts = data_process(val_dialogues)

train_df = pd.DataFrame(zip(train_texts,train_summaries), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_texts,val_summaries), columns=['Text', 'Summary'])

100%|██████████| 1746/1746 [00:00<00:00, 41848.37it/s]


In [22]:
train_df.head()

,Text,Summary
0,#상거래(쇼핑)#[sep]그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정...,"비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다."
1,#상거래(쇼핑)#[sep]kf마스크만 5부제 하는거지?[sep]응. 면마스크는 아무...,비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.
2,#상거래(쇼핑)#[sep]아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
3,#상거래(쇼핑)#[sep]칫솔사야하는데 쓱으로 살까?[sep]뭘 칫솔사는것까지 물어...,칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.
4,#상거래(쇼핑)#[sep]잠도안오네 얼릉 고구마츄 먹고싶단[sep]그게 그렇게 맛있...,잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.


In [23]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)
test_samples = Dataset.from_pandas(val_df)

print(train_data)
print(val_data)
print(test_samples)

Dataset({
    features: ['Text', 'Summary'],
    num_rows: 13995
})
Dataset({
    features: ['Text', 'Summary'],
    num_rows: 1746
})
Dataset({
    features: ['Text', 'Summary'],
    num_rows: 1746
})


In [24]:
%cd /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation1.5
!unzip /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation1.5/checkpoint-12500-20230126T012533Z-001.zip

/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation1.5
Archive:  /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation1.5/checkpoint-12500-20230126T012533Z-001.zip
  inflating: checkpoint-12500/optimizer.pt  
  inflating: checkpoint-12500/special_tokens_map.json  
  inflating: checkpoint-12500/trainer_state.json  
  inflating: checkpoint-12500/config.json  
  inflating: checkpoint-12500/scheduler.pt  
  inflating: checkpoint-12500/tokenizer.json  
  inflating: checkpoint-12500/training_args.bin  
  inflating: checkpoint-12500/rng_state.pth  
  inflating: checkpoint-12500/tokenizer_config.json  
  inflating: checkpoint-12500/pytorch_model.bin  


In [25]:
model_checkpoints = "/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation1.5/checkpoint-12500"


tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

special_words = [
                "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
                "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
                "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
                "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
                "#개인 및 관계#", "#미용과 건강#", "#상거래(쇼핑)#", "#시사/교육#", "#식음료#", 
                "#여가 생활#", "#일과 직업#", "#주거와 생활#", "#행사#","[sep]"
                ]

tokenizer.add_special_tokens({"additional_special_tokens": special_words})
model.resize_token_embeddings(len(tokenizer))

loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation1.5/checkpoint-12500/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "/content/drive/MyDrive/\u110b\u1175\u11ab\u1100\u1169\u11bc\u110c\u1175\u1102\u1173\u11bc/\u1109\u1162\u11bc\u1109\u1165\u11bc\u110b\u116d\u110b\u1163\u11a8\u1111\u1173\u1105\u1169\u110c\u1166\u11a8\u1110\u1173/Model/KoBART/checkpoint/domain_adaptation1.5/checkpoint-12500",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(ma

Embedding(30032, 768, padding_idx=3)

In [26]:
max_input = 256
max_target = 64
ignore_index = -100# tokenizer.pad_token_id

def add_ignored_data(inputs, max_len, ignore_index):
  if len(inputs) < max_len:
      pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
      inputs = np.concatenate([inputs, pad])
  else:
      inputs = inputs[:max_len]

  return inputs

def add_padding_data(inputs, max_len):
    pad_index = tokenizer.pad_token_id
    if len(inputs) < max_len:
        pad = [pad_index] *(max_len - len(inputs))
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:max_len]

    return inputs 

def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []
    bos = tokenizer('<s>')['input_ids']
    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input))
    for i in range(len(data_to_process['Summary'])):
        label_id.append(tokenizer.encode(data_to_process['Summary'][i]))  
        label_id[i].append(tokenizer.eos_token_id)   
        dec_input_id = bos
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Summary'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [27]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)   

In [28]:
model.config.max_length = 64 
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint2/domain_adaptation_ft",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=128,  # demo
    per_device_eval_batch_size=256,
    learning_rate=3e-05,
    weight_decay=0.1,
    #label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/logs2",
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch',
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?

trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13995
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 550
  Number of trainable parameters = 123884544
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,2.850300,2.268957,"{'r': 0.2127350838606856, 'p': 0.23190906516091678, 'f': 0.21435393036213726}","{'r': 0.07659862909658066, 'p': 0.08425048799813366, 'f': 0.07687426755654432}","{'r': 0.20253032064815876, 'p': 0.2211810325039865, 'f': 0.20418441020899988}"
2,2.169300,2.217841,"{'r': 0.22411012561329005, 'p': 0.23016727196316647, 'f': 0.2194727911423836}","{'r': 0.08028916462585685, 'p': 0.08352911884310392, 'f': 0.07873821984719101}","{'r': 0.2131723723631006, 'p': 0.21931893778798917, 'f': 0.20890198471554106}"
3,1.995200,2.187794,"{'r': 0.2319413218350271, 'p': 0.23637587945353622, 'f': 0.22656615181458659}","{'r': 0.08483184341162216, 'p': 0.08746028384042663, 'f': 0.08286134352786637}","{'r': 0.22060447827256163, 'p': 0.22528058271595772, 'f': 0.2156321370588832}"
4,1.883600,2.182759,"{'r': 0.23764777707652812, 'p': 0.2400076397784219, 'f': 0.231109213647846}","{'r': 0.08759881282378053, 'p': 0.08858154907656497, 'f': 0.08490910466508958}","{'r': 0.22586263805702994, 'p': 0.22825756527343039, 'f': 0.2196898534493766}"
5,1.819000,2.187689,"{'r': 0.23937299181924532, 'p': 0.2366114524456082, 'f': 0.23037818467691024}","{'r': 0.08810048613759139, 'p': 0.08763543295983901, 'f': 0.08481056989938104}","{'r': 0.2268072689435704, 'p': 0.22473912940278176, 'f': 0.21849385427469575}"


***** Running Evaluation *****
  Num examples = 1746
  Batch size = 256
Trainer is attempting to log a value of "{'r': 0.2127350838606856, 'p': 0.23190906516091678, 'f': 0.21435393036213726}" of type <class 'dict'> for key "eval/rouge-1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.07659862909658066, 'p': 0.08425048799813366, 'f': 0.07687426755654432}" of type <class 'dict'> for key "eval/rouge-2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.20253032064815876, 'p': 0.2211810325039865, 'f': 0.20418441020899988}" of type <class 'dict'> for key "eval/rouge-l" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkp

TrainOutput(global_step=550, training_loss=2.1434827215021306, metrics={'train_runtime': 1036.9908, 'train_samples_per_second': 67.479, 'train_steps_per_second': 0.53, 'total_flos': 1.0666577166336e+16, 'train_loss': 2.1434827215021306, 'epoch': 5.0})

## MLM 5epoch 결과
|Epoch|	Training Loss|	Validation Loss|	Rouge-1|	Rouge-2	|Rouge-l|
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
|1|	2.874300|	2.249203|	{'r': 0.20758037622544812, 'p': 0.22943981733211927, 'f': 0.21054159756687602}|	{'r': 0.07297490495662584, 'p': 0.08116538176534553, 'f': 0.07368258567521124}|	{'r': 0.19851691240601885, 'p': 0.21980232875257313, 'f': 0.20149710478622113}|
|2|	2.157100|	2.199063|	{'r': 0.2255228462894452, 'p': 0.2297876905136315, 'f': 0.21984014505715785}|	{'r': 0.08182770058888043, 'p': 0.08361004420312904, 'f': 0.07915242306254534}	|{'r': 0.21510963642019093, 'p': 0.21929177336209926, 'f': 0.2096580438838813}|
|3|	1.980500|	2.175380|	{'r': 0.2310013386556175, 'p': 0.2384158298030901, 'f': 0.22692351980100836}|	{'r': 0.08501624662151658, 'p': 0.08857288362715174, 'f': 0.08341287029589635}|	{'r': 0.220280398847096, 'p': 0.22763241420807084, 'f': 0.2164744319745452}|
|4|	1.867200|	2.167914	|{'r': 0.23181399374409678, 'p': 0.23856800826523908, 'f': 0.2273784389570245}|	{'r': 0.08491606940170021, 'p': 0.08815829381113957, 'f': 0.08308673534646065}|	{'r': 0.22043819377451462, 'p': 0.2268326624069312, 'f': 0.21614369073320616}|
| 5|	1.800200|	2.177052|	{'r': 0.2369867870559843, 'p': 0.23689485528311888, 'f': 0.22926557446742699}	|{'r': 0.08629378980710091, 'p': 0.08703739012403325, 'f': 0.08341910988307079}|	{'r': 0.22508887419600565, 'p': 0.22515401379332017, 'f': 0.21775992353489382}|

## MLM 4epoch 결과
|Epoch|	Training Loss|	Validation Loss|	Rouge-1|	Rouge-2	|Rouge-l|
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
|1|	2.877300|	2.254946|	{'r': 0.2131032956501339, 'p': 0.23561645599323797, 'f': 0.21608155720533975}|	{'r': 0.07671231646735555, 'p': 0.08520600116591708, 'f': 0.0774137905925165}	|{'r': 0.20284490067783315, 'p': 0.2246913178935411, 'f': 0.20584978032655288}|
|2|	2.160900	|2.198241|	{'r': 0.22352111714868933, 'p': 0.2295878342868498, 'f': 0.218546332488047}|	{'r': 0.08076735565432416, 'p': 0.08357270891284885, 'f': 0.07873271525317424}|	{'r': 0.21390727389739975, 'p': 0.220039869318428, 'f': 0.20925623578970542}|
|3|1.985000	|2.176032|	{'r': 0.23172100369748397, 'p': 0.23935265031585587, 'f': 0.2274864231191159}	|{'r': 0.08560399864919427, 'p': 0.08945147236416272, 'f': 0.08404471979619471}|	{'r': 0.22051352867590493, 'p': 0.2281068845271985, 'f': 0.21658149591553794}|
|4|	1.872300	|2.168593|	{'r': 0.2318782096934543, 'p': 0.23781262234440667, 'f': 0.22721226094244626}	|{'r': 0.08580310827860263, 'p': 0.08817205484948905, 'f': 0.08369428751186149}	|{'r': 0.22095084224131512, 'p': 0.22669040866595205, 'f': 0.21647697661776105}|
|5|1.805900	|2.177557	|{'r': 0.2385174446733895, 'p': 0.23726265051087111, 'f': 0.23034952359804212}|	{'r': 0.08771385005101427, 'p': 0.08812036818307248, 'f': 0.08469213997087471}|	{'r': 0.22656791101442533, 'p': 0.22550144426111254, 'f': 0.2187957449107862}|

## MLM 3epoch 결과
|Epoch|	Training Loss|	Validation Loss|	Rouge-1|	Rouge-2	|Rouge-l|
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
|1|	2.882700|	2.262300|	{'r': 0.2116468689829956, 'p': 0.23028595972722707, 'f': 0.21325658794551675}|	{'r': 0.07602429650076371, 'p': 0.08278062860639036, 'f': 0.07604935680660947}|	{'r': 0.20229361709677168, 'p': 0.22043942224344143, 'f': 0.20396943826003677}|
|2|	2.169600|	2.207843|	{'r': 0.22488637622916516, 'p': 0.22867499351088286, 'f': 0.21895483005380129}|	{'r': 0.08172637102067977, 'p': 0.08409900041782106, 'f': 0.07937244002871226}|	{'r': 0.21398186757611928, 'p': 0.21769743852493095, 'f': 0.20833988626781003}|
|3|	1.997200|	2.183706|	{'r': 0.22804124472898973, 'p': 0.2337916647445344, 'f': 0.22297790336980694}|	{'r': 0.08319769097154131, 'p': 0.08626665670102823, 'f': 0.08126340198718206}|	{'r': 0.21585691993638642, 'p': 0.2216942193816465, 'f': 0.21118804422083057}|
|4|	1.885100|	2.176699|	{'r': 0.23539635736661002, 'p': 0.2387313959321402, 'f': 0.22889717388106492}	|{'r': 0.08682862039330493, 'p': 0.0895481628888984, 'f': 0.08460224215028478}|	{'r': 0.22320754548511768, 'p': 0.22666050541208824, 'f': 0.21715865513305413}|
|5|	1.819800|	2.185045|	{'r': 0.23802554292378836, 'p': 0.23489716145796258, 'f': 0.22853415577947672}|	{'r': 0.08863434748927972, 'p': 0.08766887964835282, 'f': 0.08474937382644346}|	{'r': 0.2252045125431885, 'p': 0.22243045152461963, 'f': 0.21628017137197866}|

## MLM 2epoch 결과
|Epoch|	Training Loss|	Validation Loss|	Rouge-1|	Rouge-2	|Rouge-l|
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
|1|	2.850300|	2.268957|	{'r': 0.2127350838606856, 'p': 0.23190906516091678, 'f': 0.21435393036213726}|	{'r': 0.07659862909658066, 'p': 0.08425048799813366, 'f': 0.07687426755654432}	|{'r': 0.20253032064815876, 'p': 0.2211810325039865, 'f': 0.20418441020899988}|
|2|	2.169300|	2.217841|	{'r': 0.22411012561329005, 'p': 0.23016727196316647, 'f': 0.2194727911423836}	|{'r': 0.08028916462585685, 'p': 0.08352911884310392, 'f': 0.07873821984719101}|	{'r': 0.2131723723631006, 'p': 0.21931893778798917, 'f': 0.20890198471554106}|
|3|	1.995200|	2.187794|	{'r': 0.2319413218350271, 'p': 0.23637587945353622, 'f': 0.22656615181458659}|	{'r': 0.08483184341162216, 'p': 0.08746028384042663, 'f': 0.08286134352786637}|	{'r': 0.22060447827256163, 'p': 0.22528058271595772, 'f': 0.2156321370588832}|
|4	|1.883600	|2.182759	|{'r': 0.23764777707652812, 'p': 0.2400076397784219, 'f': 0.231109213647846}|	{'r': 0.08759881282378053, 'p': 0.08858154907656497, 'f': 0.08490910466508958}|	{'r': 0.22586263805702994, 'p': 0.22825756527343039, 'f': 0.2196898534493766}|
|5	|1.819000	|2.187689|	{'r': 0.23937299181924532, 'p': 0.2366114524456082, 'f': 0.23037818467691024}	|{'r': 0.08810048613759139, 'p': 0.08763543295983901, 'f': 0.08481056989938104}	|{'r': 0.2268072689435704, 'p': 0.22473912940278176, 'f': 0.21849385427469575}|

## MLM 1epoch 결과
|Epoch|	Training Loss|	Validation Loss|	Rouge-1|	Rouge-2	|Rouge-l|
|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
|1|	2.852900	|2.282979|{'r': 0.20870334940866161, 'p': 0.22574141332224712, 'f': 0.20968642204565263}|	{'r': 0.07469158408605986, 'p': 0.08094631507439717, 'f': 0.07462300103765918}|	{'r': 0.19890703810324445, 'p': 0.21530447993579893, 'f': 0.19987447466175248}|
|2|	2.187600|	2.231880|	{'r': 0.22810406925311497, 'p': 0.229448748303972, 'f': 0.22149204757876712}	|{'r': 0.08594435584174136, 'p': 0.08649026114492554, 'f': 0.08306646746281814}	|{'r': 0.21715374924689126, 'p': 0.21859347051236708, 'f': 0.21088582957265045}|
|3	|2.020100	|2.205199|	{'r': 0.23327182705080635, 'p': 0.23495003214505528, 'f': 0.2263587255011518}	|{'r': 0.0874382720562843, 'p': 0.08917472535575753, 'f': 0.08485724416834559}|	{'r': 0.22169037841217523, 'p': 0.22341220403166476, 'f': 0.21515133328400535}|
|4|	1.910100|	2.199714|	{'r': 0.23418463242350396, 'p': 0.23548265971047777, 'f': 0.22738987602083843}|	{'r': 0.08717310546397102, 'p': 0.08821211748909066, 'f': 0.08445225315382081}|	{'r': 0.22232527257056703, 'p': 0.22372248429110708, 'f': 0.2159174890798592}|
|5|	1.847000	|2.205849|	{'r': 0.239660992472629, 'p': 0.23651147363839803, 'f': 0.23056261109808704}|	{'r': 0.09039746726550081, 'p': 0.09022180754061271, 'f': 0.08693488223798096}	|{'r': 0.22742417000428333, 'p': 0.2246443100774873, 'f': 0.21886646221602946}|